In [65]:
import pandas as pd 
# list of strings 
lst_1 = ['Italy', 'US', 'Brazil', 'Australia', 'China', 'Russia', 'Germany','France', 'India', 'Spain','Argentina',
      'Japan','Turkey','Korea, South','Canada','Saudi Arabia','United Kingdom'] 
lst_2 = [-0.74, -0.66, -0.34, -0.53,-0.038, -0.29,  -0.70, -0.57, -0.44, -0.59, 0.15, -0.50, -0.56, -0.11,-0.57,
        -0.38, -0.72 ]
  
# Calling DataFrame constructor on list 
# with indices and columns specified 
df = pd.DataFrame(lst_1, columns =['Country']) 
df 
df2 = pd.DataFrame(lst_2, columns = ['Correlations'])
df_final = df.join( df2)
df_final

,Country,Correlations
0,Italy,-0.740
1,US,-0.660
2,Brazil,-0.340
3,Australia,-0.530
4,China,-0.038
5,Russia,-0.290
6,Germany,-0.700
7,France,-0.570
8,India,-0.440
9,Spain,-0.590


In [224]:
coordinate = pd.read_csv('/Users/enricoconte/Desktop/università/specialistica/1 Anno/1 Semestre/DataMan_DataViz/Progetto/Dati/covid_dataviz.csv')

corr_final = pd.merge(df_final, coordinate[['Country','Lat','Long', 'Date']], on= 'Country')
corr_final2 = corr_final[corr_final['Date'] == '2020-06-02']
corr_final2[corr_final2['Long']== -98.8139]

#corr_final.to_csv('/Users/enricoconte/Desktop/università/specialistica/1 Anno/1 Semestre/DataMan_DataViz/Progetto/Dati/correlazioni_final.csv')

,Country,Correlations,Lat,Long,Date
11795,Canada,-0.57,53.7609,-98.8139,2020-06-02


In [48]:
# Correlazione day-by-day
tabella =pd.read_csv('/Users/enricoconte/Desktop/tabella.csv')
tabella

,Unnamed: 0,Country,Symbol,Date,Close,Rate,Confirmed,Daily_confirmed
0,0,China,000001.SS,2020-06-02,2921.397949,0.204671,84161.0,7.0
1,1,Japan,^N225,2020-06-02,22325.609375,1.193065,16837.0,50.0
2,2,Germany,^GDAXI,2020-06-02,12021.280273,3.749342,183879.0,285.0
3,3,US,^NYA,2020-06-02,12046.410156,1.228294,1837374.0,20895.0
4,4,Saudi Arabia,^TASI-SR,2020-06-02,7285.230000,-0.050000,89011.0,1869.0
...,...,...,...,...,...,...,...,...
1755,1755,Argentina,^MERV,2020-01-02,41106.968750,0.000000,0.0,0.0
1756,1756,Spain,^IBEX,2020-01-02,9691.200195,0.000000,0.0,0.0
1757,1757,Canada,^GSPTSE,2020-01-02,17100.000000,0.000000,0.0,0.0
1758,1758,United Kingdom,^FTSE,2020-01-02,7604.299805,0.000000,0.0,0.0


In [52]:
#del tabella['Unnamed: 0']
tabella

,Country,Symbol,Date,Close,Rate,Confirmed,Daily_confirmed
0,China,000001.SS,2020-06-02,2921.397949,0.204671,84161.0,7.0
1,Japan,^N225,2020-06-02,22325.609375,1.193065,16837.0,50.0
2,Germany,^GDAXI,2020-06-02,12021.280273,3.749342,183879.0,285.0
3,US,^NYA,2020-06-02,12046.410156,1.228294,1837374.0,20895.0
4,Saudi Arabia,^TASI-SR,2020-06-02,7285.230000,-0.050000,89011.0,1869.0
...,...,...,...,...,...,...,...
1755,Argentina,^MERV,2020-01-02,41106.968750,0.000000,0.0,0.0
1756,Spain,^IBEX,2020-01-02,9691.200195,0.000000,0.0,0.0
1757,Canada,^GSPTSE,2020-01-02,17100.000000,0.000000,0.0,0.0
1758,United Kingdom,^FTSE,2020-01-02,7604.299805,0.000000,0.0,0.0


In [53]:
from scipy.stats import pearsonr

In [55]:
a = pearsonr(tabella['Close'],tabella['Daily_confirmed'])
a

(-0.00631952621623098, 0.7910608530729033)

In [166]:
#TEST PRODUCER
from time import sleep
from json import dumps
from kafka import KafkaProducer
from kafka import KafkaConsumer
from pymongo import MongoClient
from json import loads
import requests
import ast
import pymongo
import pandas as pd
import datetime
import time
import io
import json

#SENDING DATA TO PRODUCER
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))


consumer = KafkaConsumer(
    'alphavantage',
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group',
     value_deserializer= lambda x: loads(x.decode('utf-8')))


#import dei dati su mongo
from pymongo import MongoClient
import pymongo
client = MongoClient()
db = client.dataman
alphavantage = db.alphavantage
records_ = corr_final2.to_dict(orient = 'records')
result = db.alphavantage.insert_many(records_ )
print('Data Correctly sent to MongoDB')

Data Correctly sent to MongoDB
